In [1]:
!pip install mlflow

     ---------------------------------------- 9.7/9.7 MB 484.1 kB/s eta 0:00:00
     ---------------------------------------- 2.8/2.8 MB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 479.3 kB/s eta 0:00:00
  Using cached flask-3.1.2-py3-none-any.whl (103 kB)
     ------------------------------------ 263.9/263.9 kB 737.4 kB/s eta 0:00:00
     ------------------------------------ 147.8/147.8 kB 799.6 kB/s eta 0:00:00
     ------------------------------------ 114.9/114.9 kB 742.2 kB/s eta 0:00:00
     -------------------------------------- 77.0/77.0 kB 855.3 kB/s eta 0:00:00
     ------------------------------------ 131.2/131.2 kB 595.9 kB/s eta 0:00:00
     -------------------------------------- 56.2/56.2 kB 421.9 kB/s eta 0:00:00
     ------------------------------------ 798.3/798.3 kB 731.0 kB/s eta 0:00:00
     ---------------------------------------- 46.1/46.1 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 1.1 MB/s e

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### **IMPORT THE NECESSARY LIBRARIES**

In [2]:

import pandas as pd
import mlflow
import mlflow.sklearn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import average_precision_score


c:\Users\Arnab Bera\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### **Setup the mlflow experiment**

In [3]:
mlflow.set_experiment("Assignment_02_SMS_Spam")


2026/02/15 21:58:57 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/15 21:58:57 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/15 21:58:57 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/15 21:58:57 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/15 21:58:57 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/15 21:58:57 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/15 21:58:58 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/15 21:58:58 INFO mlflow.store.db.utils: Updating database tables
2026/02/15 21:58:58 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/15 21:58:58 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/15 21:58:58 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/02/15 21:5

<Experiment: artifact_location=('file:///c:/Users/Arnab '
 'Bera/Desktop/CMI/CMI/Applied-ML/AppliedMachineLearning-main/AppliedMachineLearning/Assignment-02/mlruns/1'), creation_time=1771172939288, experiment_id='1', last_update_time=1771172939288, lifecycle_stage='active', name='Assignment_02_SMS_Spam', tags={}>

#### **Load the dataset**

In [4]:
train_df = pd.read_csv("dataset/train.csv")
val_df = pd.read_csv("dataset/validation.csv")
test_df = pd.read_csv("dataset/test.csv")

X_train = train_df["message"]
y_train = train_df["label"]

X_val = val_df["message"]
y_val = val_df["label"]

X_test = test_df["message"]
y_test = test_df["label"]


#### **Text Vectorization**

In [5]:
vectorizer = TfidfVectorizer(stop_words="english")

X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(X_test)


#### **MODEDL-1: Logistic Regression**

In [6]:
with mlflow.start_run(run_name="Logistic_Regression"):

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train_vec, y_train)

    probs = model.predict_proba(X_test_vec)[:, 1]
    aucpr = average_precision_score(y_test, probs)

    mlflow.log_param("model_name", "LogisticRegression")
    mlflow.log_metric("AUCPR", aucpr)

    mlflow.sklearn.log_model(model, "model")

    print("Logistic Regression AUCPR:", aucpr)


2026/02/15 22:00:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\Arnab Bera\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Logistic Regression AUCPR: 0.9646995083850685


#### **MODEL-2: NB**

In [7]:
with mlflow.start_run(run_name="Naive_Bayes"):

    model = MultinomialNB()
    model.fit(X_train_vec, y_train)

    probs = model.predict_proba(X_test_vec)[:, 1]
    aucpr = average_precision_score(y_test, probs)

    mlflow.log_param("model_name", "MultinomialNB")
    mlflow.log_metric("AUCPR", aucpr)

    mlflow.sklearn.log_model(model, "model")

    print("Naive Bayes AUCPR:", aucpr)


2026/02/15 22:00:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\Arnab Bera\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Naive Bayes AUCPR: 0.9786721665112575


#### **MODEL-3: Linear SVM**

In [8]:
with mlflow.start_run(run_name="Linear_SVM"):

    model = LinearSVC()
    model.fit(X_train_vec, y_train)

    scores = model.decision_function(X_test_vec)
    aucpr = average_precision_score(y_test, scores)

    mlflow.log_param("model_name", "LinearSVC")
    mlflow.log_metric("AUCPR", aucpr)

    mlflow.sklearn.log_model(model, "model")

    print("Linear SVM AUCPR:", aucpr)


2026/02/15 22:01:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\Arnab Bera\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Linear SVM AUCPR: 0.9851493315631116


#### **Show all the model**

In [11]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name("Assignment_02_SMS_Spam")

runs = client.search_runs(experiment.experiment_id)

print("\nAll Model AUCPR Scores:\n")

for run in runs:
    print(run.data.params["model_name"], " :AUCPR:", run.data.metrics["AUCPR"])



All Model AUCPR Scores:

LinearSVC  :AUCPR: 0.9851493315631116
MultinomialNB  :AUCPR: 0.9786721665112575
LogisticRegression  :AUCPR: 0.9646995083850685
